<a href="https://colab.research.google.com/github/yuvalgrossman/EquSolve/blob/master/HASY_transfer_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tarfile
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
import pdb
import time
from PIL import Image

!wget 'https://zenodo.org/record/259444/files/HASYv2.tar.bz2?download=1'
my_tar = tarfile.open('HASYv2.tar.bz2?download=1')
my_tar.extractall() # specify which folder to extract to
my_tar.close()
!git clone https://github.com/yuvalgrossman/EquSolve

from EquSolve.Utils.mapper import mapper
from EquSolve.Classifier.HASYDataLoader import ExampleDataset

--2020-08-26 07:46:44--  https://zenodo.org/record/259444/files/HASYv2.tar.bz2?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34597561 (33M) [application/octet-stream]
Saving to: ‘HASYv2.tar.bz2?download=1’

HASYv2.tar.bz2?down 100%[===================>]  32.99M  57.6MB/s    in 0.6s    

2020-08-26 07:46:45 (57.6 MB/s) - ‘HASYv2.tar.bz2?download=1’ saved [34597561/34597561]

Cloning into 'EquSolve'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (91/91), done.
remote: Total 103 (delta 36), reused 29 (delta 5), pack-reused 0
Receiving objects: 100% (103/103), 9.23 MiB | 11.08 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [38]:
lr = 0.01
momentum = 0.9
n_epochs = 10
batch_size = 32

In [39]:
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

class HASYDataset(Dataset):
    def __init__(self, config, csv_df,transforms):
        self.config = config
        self.data = csv_df
        self.transforms = transforms

    def __len__(self):
        return len(self.data.index)

    def __getitem__(self, idx):
        y = self.data.loc[idx, 'symbol_id']
        img_path = self.config['data_path'] + self.data.loc[idx, 'path']
        img = Image.open(img_path) # load image in PIL format
        X = self.transforms(img)   # apply transforms: resize-> tensor-> normalize
        X_reshape = X[0].unsqueeze(-1).transpose(2,0) # reshape to [1,28,28]
        return (X_reshape, y)

    def plotitem(self, idx):
        y = self.data.loc[idx, 'latex']
        # X = plt.imread(self.config['data_path'] + self.data.loc[idx, 'path'][6:])[:, :, 0]
        X = plt.imread(self.config['data_path'] + self.data.loc[idx, 'path'])[:, :, 0]

        plt.imshow(X)
        plt.title(y)
        print('img size {}'.format(X.shape))
        # print((X[:,:,2]==X[:,:,0]).all())
        # print((X[:,:,2]==X[:,:,1]).all())

In [40]:
meta_data = pd.read_csv('hasy-data-labels.csv')
sym_list = ['1','2','3','4','5','6','7','8','9','\\alpha','=','+','-','\\pi','A','X','\\cdot']
all_df = mapper(meta_data,sym_list)
all_df.latex.value_counts()

\alpha    2601
\pi       1533
\cdot      755
A          159
2          124
8          121
3          120
-          118
1          118
6          100
+           90
9           90
5           78
7           75
4           61
X           54
Name: latex, dtype: int64

In [41]:

n_classes = len(sym_list)
train_split = 0.8
config = {'data_path':''}
transform = transforms.Compose([transforms.Resize([28,28]),                                
                      transforms.ToTensor(),
                      transforms.Normalize(0.5,0.5),
                      ])
dataset = HASYDataset(config,all_df,transform)

train_size = int(train_split * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [43]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))); 
        x = self.pool(F.relu(self.conv2(x)));
        x = x.view(-1, 16 * 4 * 4); 
        x = F.relu(self.fc1(x)); 
        x = F.relu(self.fc2(x));
        x = self.fc3(x);
        return x


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
net = Net()#.to(device)

net

cuda


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [44]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=momentum)
# if device==torch.device('cpu'):
  # net.load_state_dict(torch.load('EquSolve/MNISTnet.pth', map_location=lambda storage, loc: storage)) # map to CPU
# elif device==torch.device('cuda'):
net.load_state_dict(torch.load('EquSolve/MNISTnet.pth'))

for param in net.parameters():
  print(param.shape)
  # param.requires_grad = False # freeze model weights

net.fc3 = nn.Linear(84, n_classes)
net

torch.Size([6, 1, 5, 5])
torch.Size([6])
torch.Size([16, 6, 5, 5])
torch.Size([16])
torch.Size([120, 256])
torch.Size([120])
torch.Size([84, 120])
torch.Size([84])
torch.Size([10, 84])
torch.Size([10])


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=17, bias=True)
)

In [45]:
net.conv1.weight.device

device(type='cpu')

In [46]:
net = net.to(device)
net.conv1.weight.device

device(type='cuda', index=0)

In [47]:
def test():
  correct = 0
  total = 0
  with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images.to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels.to(device)).sum().item()
  print('model accuracy: %d %%' % (100 * correct / total))

In [48]:
net.train()
for epoch in range(n_epochs):  # loop over the dataset multiple times
    t = time.time()
    for i, data in enumerate(train_loader, 0):
        
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.to(device))
        loss = criterion(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
    
    # print statistics
    print('epoch {} loss:{:.3f} time:{:.3f}'.format(epoch + 1, loss.item(), time.time()-t))
    test()
    

print('Finished Training')


epoch 1 loss:0.176 time:2.970
model accuracy: 89 %
epoch 2 loss:0.094 time:2.890
model accuracy: 93 %
epoch 3 loss:0.061 time:2.842
model accuracy: 95 %
epoch 4 loss:0.056 time:2.954
model accuracy: 95 %
epoch 5 loss:0.016 time:2.902
model accuracy: 96 %
epoch 6 loss:0.009 time:2.979
model accuracy: 96 %
epoch 7 loss:0.009 time:2.915
model accuracy: 96 %
epoch 8 loss:0.006 time:2.971
model accuracy: 96 %
epoch 9 loss:0.012 time:2.871
model accuracy: 96 %
epoch 10 loss:0.040 time:2.963
model accuracy: 95 %
Finished Training


In [14]:
net.conv1.weight.device

device(type='cuda', index=0)